 # 0. 딥러닝 실습 환경 소개

In [1]:
import tensorflow as tf

### 1) 하이퍼파라미터 설정

In [2]:
EPOCHS = 5

### 2) 네트워크 구조 정의

In [3]:
#keras를 상속받아서 사용
class MyModel(tf.keras.Model):
    def __init__(self) : #init에서 앞으로 사용할 모든 layer를 정의
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape = (28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation = 'sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation = 'softmax')
    
    def call(self, x) : #call에서 각 layer의 연결성을 구성
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

### 3) 학습loop 정의

In [23]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy) : #다양한 입력값을 받아서
    with tf.GradientTape() as tape : 
        predictions = model(images) #model에 input image을 넣어서 출력을 내고
        loss = loss_object(labels, predictions)#출력과 정답을 비교해서
    gradients = tape.gradient(loss, model.trainable_variables)        #loss, gradient를 점차 개선
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)



### 4) test loop 정의 : 학습 loop 과 거의 유사한데, gradient를 만드는 부분만 빠져있는 것이다.

In [21]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

### 5) 실질적인 학습 : 학습 데이터셋 가져오고, 정리

In [13]:
#import and organize dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### 6) 네트워크 생성

In [14]:
model = MyModel()

### 7) 손실함수, 최적화 알고리즘 정의

In [15]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### 8)알고리즘 평가지표 설정 

In [16]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')





### 9) 알고리즘 학습, 테스트 루프

In [ ]:
for epoch in range(EPOCHS) : 
    for images, labels in train_ds : 
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
        for test_images, test_labels in test_ds : 
            test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
            
        template = 'Epoch : {}, Loss : {}, Accuracy : {}, Test Loss : {}, Test Accuracy : {}'
        print(template.format(epoch + 1, 
                             train_loss.result(), 
                             train_accuracy.result() *100, 
                             test_loss.result(), 
                             test_accuracy.result() *100))

Epoch : 1, Loss : 2.5846405029296875, Accuracy : 3.125, Test Loss : 2.459831714630127, Test Accuracy : 10.40999984741211
Epoch : 1, Loss : 2.580331802368164, Accuracy : 7.8125, Test Loss : 2.4214465618133545, Test Accuracy : 11.045000076293945
Epoch : 1, Loss : 2.502699851989746, Accuracy : 10.416666030883789, Test Loss : 2.3896024227142334, Test Accuracy : 12.373332977294922
Epoch : 1, Loss : 2.486110210418701, Accuracy : 10.9375, Test Loss : 2.3609459400177, Test Accuracy : 14.255001068115234
Epoch : 1, Loss : 2.4454658031463623, Accuracy : 11.875, Test Loss : 2.3347837924957275, Test Accuracy : 16.507999420166016
Epoch : 1, Loss : 2.4160077571868896, Accuracy : 13.020833015441895, Test Loss : 2.310385227203369, Test Accuracy : 18.84833335876465
Epoch : 1, Loss : 2.3808157444000244, Accuracy : 15.625, Test Loss : 2.2873430252075195, Test Accuracy : 20.928571701049805
Epoch : 1, Loss : 2.341986656188965, Accuracy : 19.53125, Test Loss : 2.2657792568206787, Test Accuracy : 22.700000762